# Behavioral Data Analysis - Change Detection Task

Notes: Using manual git uploads for now. Consider installing widget
Integer Division 'requires' python 3. Cowan's K otherwise null. 

## Example Data

In [566]:
#!/usr/bin/env python3

# Emulate py3
from __future__ import (absolute_import, division,
                            print_function, unicode_literals)
from builtins import *
# Integer division may result in null if not run with Python > 3
   
import sys
print(sys.version)
#if sys.version[0][0] < 3:
    #del #future stuff....
    

2.7.12 |Anaconda 4.2.0 (32-bit)| (default, Jun 29 2016, 11:42:13) [MSC v.1500 32 bit (Intel)]


In [567]:
''' Prepare Data for Import '''




''' Analysis of Behavioral Data

Example input file (Line breaks represent CSV)

0	 ,
1	23,
2	f,
3	Subliminal CDT July 2016,
4	0,
5	lo,
6	3,
7	non-match,
8	left,
9	999999,
10	FA,
11	592,
12	"(['BLACK', 'RED', 'BLUE'],
		['BLACK', 'BLACK', 'GREEN'])",
13	"([-4.833454718581354, -2.8824553987581445, -2.3081588355080522],
		[-0.0037853209621609984, -2.8078914856435437, -0.31239484357072067],
		[3.86559432251519, 4.653214540852498, 1.83731757692552],
		[-1.3117916221511483, 1.866184035822822, -3.2871611286588642])",
14	"(['BLACK', 'RED', 'GREEN'],
		['BLACK', 'BLACK', 'GREEN'])"

'''

import numpy as np
import numpy.ma as ma
import pandas as pd
import fnmatch
import inspect
from os import getcwd, path, walk

print(60 * '=')
#
#
# Experiment Configuration #
#
#

items = [2, 3, 4, 5]

current_dir = getcwd() + path.sep + 'data' + path.sep

print('Current directory is ' + current_dir)

# Stackoverflow Recipe
f = []
for (dirpath, dirname, filename) in walk(current_dir):
    f.extend(filename)
    break
    
#files = pd.Series(f)
#print(files)


#from itertools import iter
# Initialize Data Accumulation
iter_files = iter(f) #np.nditer(f)
current_iteration = 0 # Subject ID

#name = 'ewabre_py_2016-05-16 13-41-12.756000.log'
# TODO WARNING Notice the error of the header for coordinates and colors


Current directory is D:\analysis\pomo\notebook\alpha\behavior\data\


In [602]:
current_iteration += 1
filename = iter_files.next()
print('Current file is {}...'.format(filename))

filename = path.join(current_dir, filename)

file_data = np.genfromtxt(
	filename,
	dtype = None,
	names = True,
	delimiter = ',',
	missing_values = (999999),
	usecols = (1,2,4,5,6,7,8,9,10) # TODO Ignoring stimulus data for now
)

d = file_data.ndim
file_data = np.reshape(file_data,(file_data.size, 1))

print('Dim reshaped from {} to {}'.format(d, file_data.ndim))

#print('File Preview: ')
# Print the first 4 rows
#print(file_data[0:4])

print(60 * '-')
# Remove illegal responses
l = len(file_data)
file_data = file_data[file_data['RT'] < 7000]
#print(file_data)
print('Missing Values Removed: {}'.format(l - len(file_data)))
#print(file_data[0:4])
print(60 * '^')

# Compute Signal Detection Theory (SDT) measures
print('Level: Subject')
r = file_data['Response']
print('{} responses'.format(len(r)))
hit = len(r[r == b'HIT'])
miss = len(r[r == b'MISS'])
fa = len(r[r == b'FA'])
cr = len(r[r == b'CR'])
total = hit + miss + fa + cr
assert len(r) == total

print('Total Valid Trials {}'.format(total))
hitrate = 100 * hit / total
farate = 100 * fa / total
print('Hits: {}\nMisses: {}\nFalse Alarms: {}\nCorrect Responses: {}'.format(hit,miss,fa,cr))

# Rounding
decimals = 2

print('Hit{2}: {0}%\nFalse Alarm{2}: {1}%\n[of valid responses]'.format(round(hitrate, decimals),round(farate, decimals),'rate'))

print('Level: Per Item in Subject')
print('Items: {}'.format(items))

#  Compute Cowan's K for all items
 
file_data = pd.DataFrame(file_data)

#print(file_data)

data = file_data

'''a'''

'''TODO Groupby Subject ID... Accumulate'''
agg_responses_per_items = data.groupby(['Items', 'Response']).aggregate(pd.DataFrame.count)['Response']
agg_responses_per_items

from sys import modules, version
# Guardian clause for integer division
# ON ERROR: Run the initial notebook imports again, located at the top of the file.
assert ('__future__' in sys.modules) or (sys.version[0][0] > 2)

# For each item get total and individual SDT values

a = agg_responses_per_items # Shorthand

#agg_responses_per_items.loc[:,['CR']]
#ai = np.nditer(a, flags=['multi_index'])
# no clue what setting this flag or not does. preserves multi_index? #ainoflag = np.nditer(a)

# Helper function
fun = lambda x: np.asarray(x)

#
# Signal Detection Measures
#
#
#
#

totali = fun([sum(a.loc[i,:]) for i in items])
hi = fun(a.loc[:, ['HIT']])
fai = fun(a.loc[:, ['FA']])
cri = fun(a.loc[:, ['CR']])
mi = fun(a.loc[:, ['MISS']])

# Rounding precision
decimals = 2 

# False Alarm Rate (False Positive Rate)
# NTS: Might be iteresting to look at the ratios here... assuming the limit corresponds to a quanta - i.e. a limited resource (limes being the quantum) - that can be partially distributed....
print('FAs: {}'.format(np.round(fai / totali * 100, decimals)))


# The original
# (H + CR - 1) * N

# Vector of items
N = fun(items)

# Because we had fun, we can by means of symbolical math compute Cowan's K thus
ck1 = np.round( ( (hi + cri - 1) / totali ), decimals)
ck2 = np.round( ( N * (hi - fai) / totali ), decimals)
k_zero = np.round( (ck1 + ck2) / 2 , decimals)
global_total = sum(a)


# Stratified samples of n trials weighted by evidence per item
# Compute the item percentage of the total sample
strata_weights = totali / global_total
# Compute the mean percentage
mean_weight = np.mean(strata_weights)
# Center the weights around the mean percentage by subtracting the mean
mean_centered_weights = strata_weights - mean_weight

print(strata_weights)
print(mean_centered_weights)
ck_error = 1 - (ck1 + ck2) / 2

new_weights = mean_centered_weights + 1
print(new_weights)


# Just checking if using numpy results in 32-bit mess or if 64bit precision is kept.
# Great. Test passed.
#print('{}{}'.format(type(new_weights[0]), type(strata_ck1[0])))

# Finally, weigh Cowan's k+ and k-
strata_ck1 = np.round(new_weights * ck1, decimals)
strata_ck2 = np.round(new_weights * ck2, decimals)

# For each item
ones = fun(len(items) * [1])

#lk_error = (ck2 - ck1) / 2

# Print debug information
print('Per Item (2, 3, 4, 5)')
print('Total: {}\nHits: {}\nFalse Alarms: {}'.format(totali, hi, fai))
print('Cowan''s k+: {}'.format(ck1))
print('Cowan''s k-: {}'.format(ck2))
print('Cowan''s k0: {}'.format(k_zero))
print(60 * '#')
print('{}{}{}{}{}{}'.format(ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2))
print(60 * '#')
print('k error: {}'.format(ck_error))
print('k+ weighted by coefficents (stratified sampling): {}'.format(strata_ck1))
print('k- weighted by coefficents (stratified sampling): {}'.format(strata_ck2))

if current_iteration == 1:
    big_data = pd.DataFrame(data=[ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2])
else:
    next_data = pd.DataFrame(data=[ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2])

try:
    big_data = pd.DataFrame.append(big_data, next_data)
except StopIteration:
    print('Reached STOPITERATION')
finally:
    pass

big_data.T



StopIteration: 

AttributeError: 'numpy.int64' object has no attribute 'val'

In [603]:
big_data.T

,0,1,2,3,4,5,0,1,2,3,...,2,3,4,5,0,1,2,3,4,5
0,0.92,0.82,0.87,0.130,0.91,0.81,0.97,0.91,0.94,0.060,...,0.81,0.190,0.96,0.76,0.90,0.71,0.80,0.195,0.89,0.70
1,0.80,0.87,0.84,0.165,0.82,0.89,0.94,1.38,1.16,-0.160,...,0.84,0.160,0.62,0.71,0.81,1.03,0.92,0.080,0.83,1.05
2,0.84,1.39,1.12,-0.115,0.84,1.38,0.85,1.48,1.16,-0.165,...,1.06,-0.065,0.90,1.39,0.70,1.01,0.86,0.145,0.69,1.00
3,0.75,1.57,1.16,-0.160,0.74,1.55,0.84,1.79,1.32,-0.315,...,1.23,-0.230,0.83,1.79,0.66,0.67,0.66,0.335,0.66,0.67


In [624]:
xw = pd.io.api.ExcelWriter('data1.xlsx')
big_data.T.to_excel(xw,'Sheet1')
xw.save()

## Response Count

### Compute Cowan's *k*

Cowan's
\begin{equation*}
k_{-} = N * (h + c - 1)
\end{equation*}

Cowan's
\begin{equation*}
k_{+} = N * (h - f)
\end{equation*}


In [569]:
# Generate Labels

ITEMS_ = 0
SDT_ = 1

a = agg_responses_per_items
step_item = len(a.index.levels[ITEMS_])
len_agg = len(a.index.labels[ITEMS_])

#start = 0
#stop = len_agg

#item_mask = [i for i in range(start, stop, step_item)]
#print(item_mask)

#sdt_mask = a.index.labels[SDT_]
## DEBUGGABLE TODO --- MAY not return for other items than a quadruple, e.g. [2,3,4,5]
#[i for i in step_item * range(start, step_item)] #replace step_item
#print(sdt_mask)

multi_label = ''

sep = ', '
for i in a.index.levels[SDT_]:
    for j in a.index.levels[ITEMS_]:
        multi_label += '{}-{}{}'.format(j, i, sep)

# Trim last separator
multi_label = multi_label[0:-len(sep)]

print(multi_label)

2-CR, 3-CR, 4-CR, 5-CR, 2-FA, 3-FA, 4-FA, 5-FA, 2-HIT, 3-HIT, 4-HIT, 5-HIT, 2-MISS, 3-MISS, 4-MISS, 5-MISS


## TODO: Outlier Removal

## Median Response Times

In [570]:
agg_median_rt_per_items = data.groupby(['Items', 'Response']).aggregate(pd.DataFrame.median)['RT']
agg_median_rt_per_items

Items  Response
2      CR          449.0
       FA          428.0
       HIT         388.5
       MISS        429.5
3      CR          470.5
       FA          457.0
       HIT         415.0
       MISS        472.0
4      CR          480.0
       FA          421.0
       HIT         436.0
       MISS        469.0
5      CR          477.0
       FA          494.0
       HIT         441.5
       MISS        466.0
Name: RT, dtype: float64

# MAD

In [571]:
agg_mad_rt_per_items = data.groupby(['Items', 'Response']).aggregate(pd.DataFrame.mad)['RT']
agg_mad_rt_per_items

Items  Response
2      CR           58.162974
       FA           95.333333
       HIT          56.668269
       MISS         80.968750
3      CR           59.967705
       FA           56.231009
       HIT          56.982022
       MISS         58.312925
4      CR           51.043159
       FA           61.152355
       HIT          76.826978
       MISS         57.515571
5      CR           63.691106
       FA          126.451524
       HIT          85.273554
       MISS         56.144044
Name: RT, dtype: float64

</br></br></br></br></br>
*The End (for now...)*